In [5]:
import streamlit as st
import altair as alt
import polars as pl
import numpy as np
import sys
import plotly.express as px
import pandas as pd

In [ ]:
sys.path.insert(0, '../Eleicoes/')

In [6]:
pd.__version__

'2.1.1'

In [ ]:
import standards as sdt_f

In [ ]:
poll_18:str = "eleicao18_turno_01.parquet"
poll_22:str = "eleicao22_turno_01.parquet"
parties:str = "partidos_br.parquet"

In [ ]:
def get_df(url):
    return pl.read_parquet(source=url)

In [ ]:
def get_regioes():
    return get_df("https://raw.githubusercontent.com/perferctstorm/DiscursoOdioEleicoes/main/Dados/Eleicoes/regioes.parquet")     

In [ ]:
def get_eleicao_18():
    return get_df(f"https://raw.githubusercontent.com/perferctstorm/DiscursoOdioEleicoes/main/Dados/Eleicoes/{poll_18}")     

In [ ]:
def get_eleicao_22():
    return get_df(f"https://raw.githubusercontent.com/perferctstorm/DiscursoOdioEleicoes/main/Dados/Eleicoes/{poll_22}")     

In [ ]:
def get_partidos():
    return get_df(f"https://raw.githubusercontent.com/perferctstorm/DiscursoOdioEleicoes/main/Dados/Eleicoes/{parties}")     

In [ ]:
def get_municipios():
    return get_df(f"https://raw.githubusercontent.com/perferctstorm/DiscursoOdioEleicoes/main/Dados/Eleicoes/municipios.parquet")     

In [ ]:
def get_capitais():
    return get_df(f"https://raw.githubusercontent.com/perferctstorm/DiscursoOdioEleicoes/main/Dados/Eleicoes/capitais.parquet")     


In [ ]:
#disabilita o limite de 5.000 para processamento imposto pelo altair
alt.data_transformers.disable_max_rows()

In [ ]:
df_regioes = get_regioes()
df_regioes.limit(1)

In [ ]:
df_poll_18:pl.DataFrame = get_eleicao_18()
df_poll_18 = df_poll_18.filter(pl.col("CD_CARGO")==1)
df_poll_18 = df_poll_18.drop("NR_ZONA")
df_poll_18.limit(1)

In [ ]:
df_poll_22:pl.DataFrame = get_eleicao_22()
df_poll_22 = df_poll_22.filter(pl.col("CD_CARGO")==1)
df_poll_22 = df_poll_22.drop("NR_ZONA")
df_poll_22.limit(1)

In [ ]:
df_municipios = get_municipios()
df_capitais = get_capitais()

In [ ]:
df_partidos = get_partidos()

In [ ]:
df_colors = sdt_f.parties_colors(df_partidos) #define uma cor para cada partido político baseado na sua ideologia

In [ ]:
total_votos_pres_18:np.int32 = (
    df_poll_18
    .filter(pl.col("CD_CARGO")==1)
    .get_column("QT_VOTOS_VALIDOS").sum()
)
total_votos_pres_18

In [ ]:
cruzamento:list[str] = ["PT 2022 x PL 2022", "PT 2022 x PSL 2018", "PT 2022 x PT 2018", 
                        "PL 2022 x PSL 2018", "PL 2022 x PT 2018"]

In [ ]:
cols_list:list[str] = df_poll_18.columns
cols_list.remove("QT_VOTOS_VALIDOS")

In [ ]:
df_poll_18 = df_poll_18.group_by(cols_list).sum()
df_poll_22 = df_poll_22.group_by(cols_list).sum()

In [ ]:
df_poll_18.filter( (pl.col("NM_MUNICIPIO")=="FORTALEZA") & (pl.col("SG_UF")=="CE")).get_column("SG_PARTIDO").value_counts().filter(pl.col("count")>1)

In [ ]:
df_poll_18 = df_poll_18.with_columns(
    pl.col("QT_VOTOS_VALIDOS").sum().over("CD_MUNICIPIO").alias("TOTAL_VOTOS_MUNIC"),
    (pl.col("QT_VOTOS_VALIDOS")/ pl.col("QT_VOTOS_VALIDOS").sum().over("CD_MUNICIPIO")).alias("PCT_VOTOS_MUNIC"),
)

In [ ]:
df_poll_22 = df_poll_22.with_columns(
    pl.col("QT_VOTOS_VALIDOS").sum().over("CD_MUNICIPIO").alias("TOTAL_VOTOS_MUNIC"),
    (pl.col("QT_VOTOS_VALIDOS")/ pl.col("QT_VOTOS_VALIDOS").sum().over("CD_MUNICIPIO")).alias("PCT_VOTOS_MUNIC"),
)

In [ ]:
def votting_by_region(
  df:pl.DataFrame, 
  df_municipios:pl.DataFrame,
  cargo:list[str]=[],
  partidos:list[str]=[],
  group_by_cols:list[str]=["ANO_ELEICAO", "NM_REGIAO"])->pl.DataFrame:
    
    if partidos:
        df = df.filter((pl.col("SG_PARTIDO").is_in(partidos)))
        
    df = (df
            .filter( pl.col("CD_CARGO").is_in( cargo ) )
            .group_by(group_by_cols)
            .agg(pl.col("QT_VOTOS_VALIDOS").sum())           
        )
    df = df.with_columns(pl.lit("B").alias("TIPO")) if partidos else df.with_columns(pl.lit("A").alias("TIPO"))
        
    return df

In [ ]:
df_res_part_reg = votting_by_region(
    df=df_poll_22, 
    df_municipios=df_municipios,
    cargo=list([1]),
    partidos=["PT"]
)

In [ ]:
df_res_reg = votting_by_region(
    df=df_poll_22, 
    df_municipios=df_municipios,
    cargo=list([1]),
    partidos=[]
)

In [ ]:
df_resume = pl.concat([df_res_reg, df_res_part_reg])

In [ ]:
'''
  Resumo dos mapas 

  df_totais:pl.DataFrame -> votos totais por região
  year:int -> ano da eleição
'''
def map_resume(
  df_totais:pl.DataFrame, 
  color:str=""
)->alt.vegalite.v5.api.Chart:
  color = alt.condition(
    alt.datum.TIPO == 'A',
      alt.value('#FCC053'),
      alt.value(color)
  )
  ##########################
  ## Barras valore totais ##
  ##########################
  base = (
      alt.Chart(df_totais)
      .mark_bar(strokeWidth=.5, stroke="#fff", cornerRadius=4, size=8)
      .encode(
        x=alt.X('NM_REGIAO:N', title="", axis=alt.Axis(labelAngle=-90)),
        y=alt.Y('QT_VOTOS_VALIDOS:Q', title="Votação", axis=alt.Axis(format="s")),
        xOffset="TIPO:N",
        color=color,
        tooltip=[
            alt.Tooltip('NM_REGIAO:N', title="Região"),
            alt.Tooltip('QT_VOTOS_VALIDOS:Q', format=",d", title="Votação"),
        ],
        text=alt.Text('sum(QT_VOTOS_VALIDOS):Q', format=",d"),
      )
  )

  return alt.layer(
      base,
      (
          base
          .mark_text(fontSize=10, yOffset=-25, angle=270)
          .encode(color=alt.value("#000"))
      )
  )

In [ ]:
#["#C54B53", "#347DB6"]
map_resume(
    df_totais=df_resume, color="#C54B53"
).configure_title(
    anchor="middle"
).configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
).properties(width=200, height=120)

In [ ]:
#https://plotly.com/python/choropleth-maps/#geojson-with-featureid
#https://www.reddit.com/r/gis/comments/kt50l0/python_gis_how_to_reduce_memory_size_of_geojson/?rdt=56521

In [ ]:
def make_choropleth(
    input_df, input_id, input_column, input_color_theme
):
    choropleth = px.choropleth(input_df, locations=input_id, color=input_column, locationmode="USA-states",
                               color_continuous_scale=input_color_theme,
                               range_color=(0, max(df_selected_year.population)),
                               scope="usa",
                               labels={'population':'Population'}
                              )
    choropleth.update_layout(
        template='plotly_dark',
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=0, b=0),
        height=350
    )
    return choropleth

In [ ]:
#df_poll_18

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})

import plotly.express as px

fig = px.choropleth(df, geojson=counties, locations='fips', color='unemp',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           #scope="brazil",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
import plotly as plt
import plotly.express as px
import json
from urllib.request import urlopen
import pandas as pd

with urlopen('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson') as response:
    Brazil = json.load(response) # Javascrip object notation 

state_id_map = {}
for feature in Brazil ['features']:
    feature['id'] = feature['properties']['name']
    state_id_map[feature['properties']['sigla']] = feature['id']

soybean = pd.read_csv('https://raw.githubusercontent.com/nayanemaia/Dataset_Soja/main/soja%20sidra.csv')

fig = px.choropleth(
 soybean, #soybean database
 locations = 'Estado', #define the limits on the map/geography
 geojson = Brazil, #shape information
 color = 'Produção', #defining the color of the scale through the database
 hover_name = 'Estado', #the information in the box
 hover_data =["Produção","Longitude","Latitude"],
 title = "Produtivida da soja (Toneladas)", #title of the map
 animation_frame = 'ano' #creating the application based on the year
)
fig.update_geos(fitbounds = "locations", visible = False)
fig.show()

In [ ]:
fig = px.choropleth_mapbox(
 soybean, #soybean database
 locations = "Estado", #define the limits on the map/geography
 geojson = Brazil, #shape information
 color = "Produção", #defining the color of the scale through the database
 hover_name = "Estado", #the information in the box
 hover_data =["Produção","Longitude","Latitude"],
 title = "Produtivida da soja (Toneladas)", #title of the map
 mapbox_style = "carto-positron", #defining a new map style 
 center={"lat":-14, "lon": -55},#define the limits that will be plotted
 zoom = 3, #map view size
 opacity = 0.5, #opacity of the map color, to appear the background
 animation_frame = "ano" #creating the application based on the year
)
fig.show()